<a href="https://colab.research.google.com/github/posi2/Disaster_Tweets/blob/main/Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dowload Kaggle Data

First verify it kaggle api token https://www.kaggle.com/general/74235
dowload competition data

In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"positivepc","key":"04ae5ee64fdda269fa5acad848c51478"}'}

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
!kaggle competitions download -c nlp-getting-started

train.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!ls

'kaggle (1).json'   sample_data		    test.csv
 kaggle.json	    sample_submission.csv   train.csv


## Load and Analyze data

In [6]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [7]:
df =  pd.read_csv("train.csv")
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [10]:

!ls

'kaggle (1).json'   sample_data		    test.csv
 kaggle.json	    sample_submission.csv   train.csv


### Approach

1. Consider only text column
    1. make problem similar to the spam detection
    2. use TF-IDF then apply simple ML models
    3. Make submissiom
    Optimzation:
    1. Preprocessing such as lower case, stopword removal
2. Consider only text column
    1. Try deep learning models
    2. Try transformer

## Modelling with sklearn pipeline

In [11]:
X,y = df['text'], df['target']

In [12]:
 X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)

In [13]:
pipeline = Pipeline([
                     ('tf-idf', TfidfVectorizer()),
                     ('reg', LogisticRegression())
                     ])

In [14]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tf-idf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('reg',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scalin

In [15]:
pipeline.score(X_test,y_test)

0.7957977675640184

# Prediction

In [22]:
df_test = pd.read_csv("test.csv")
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [23]:
test = df_test['text']

In [24]:
"""test_pipe = Pipeline(
    [
     ('tf-idf', TfidfVectorizer())
    ]
)"""

In [25]:
#test_pipe.fit(test)

In [27]:
preds = pipeline.predict(test)

In [29]:
df_sub = pd.DataFrame({'id': df_test.id, 'target' : preds} )

In [30]:
df_sub.to_csv('submission.csv', index=False)

## Submission

In [31]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "First Submission using sklearn Pipeline with single var(text)"

100% 22.2k/22.2k [00:00<00:00, 85.3kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets